In [1]:
from itertools import accumulate, combinations, permutations, product
from functools import partial
import numpy as np

In [2]:
def transpose(x, n):
    return x[-n:] + x[:-n]

class Necklace(tuple):
    def __eq__(self, other):
        for idx in range(len(self)):
            if tuple.__eq__(transpose(self, idx), other):
                return True
        return False

In [3]:
def join(a, b):
    assert a[-1] == b[0]
    return a[:-1] + b

In [4]:
def get_harmonic_scales(seconds, thirds, edo=31):
    scales = []
    pairs = [list(p) for p in product(seconds, seconds) if sum(p) in thirds]
    matching_pairs = [join(*s) for s in product(pairs, pairs) if s[0][-1] == s[1][0]]

    for _ in range(6):
        matching_pairs = [join(*s) for s in product(matching_pairs, pairs) if s[0][-1] == s[1][0]]
        matching_pairs = [s for s in matching_pairs if sum(s) <= edo]
        for s in matching_pairs:
            if sum(s) == edo and [s[-1], s[0]] in pairs:
                if not any([Necklace(s) == Necklace(d) for d in scales]):
                    scales.append(s)
        matching_pairs = [s for s in matching_pairs if sum(s) < edo]
    
    return scales

In [5]:
def delta_to_cents(deltas, edo=31):
    return list(accumulate([(12 / edo) * d for d in deltas]))

In [6]:
def count_n_5ths(scale, fifth_step=18):
    n5ths = 0
    for i in range(len(scale)):
        if sum(transpose(scale, i)[:4]) == fifth_step:
            n5ths += 1

    return n5ths

In [7]:
# Only for 31-EDO
def distance_from_major(scale):
    major = tuple(accumulate([3, 5, 5, 3, 5, 5, 5], initial=0))
    distance = 99999
    for i in range(len(scale)):
        ts = transpose(scale, i)
        ts = tuple(accumulate(ts, initial=0))
        d = sum([abs(x - y) for x, y in zip(major, ts)])
        if d < distance:
            distance = d
    
    return distance

In [46]:
allowed_seconds = [2, 3, 4, 5, 7]
allowed_thirds = [7, 8, 9, 10, 11]
scales = get_harmonic_scales(allowed_seconds, allowed_thirds, edo=31)
scales.sort(key=count_n_5ths, reverse=True)

In [47]:
print(len(scales))
for s in scales:
    print(str(count_n_5ths(s)) + ': ' + str(s) + ' => ' + str([f'{p:.2f}' for p in delta_to_cents(s)]))

214
6: [3, 5, 5, 3, 5, 5, 5] => ['1.16', '3.10', '5.03', '6.19', '8.13', '10.06', '12.00']
5: [3, 5, 5, 4, 4, 5, 5] => ['1.16', '3.10', '5.03', '6.58', '8.13', '10.06', '12.00']
5: [4, 4, 5, 4, 4, 5, 5] => ['1.55', '3.10', '5.03', '6.58', '8.13', '10.06', '12.00']
5: [4, 4, 5, 4, 5, 4, 5] => ['1.55', '3.10', '5.03', '6.58', '8.52', '10.06', '12.00']
4: [2, 5, 4, 4, 5, 4, 7] => ['0.77', '2.71', '4.26', '5.81', '7.74', '9.29', '12.00']
4: [2, 7, 4, 5, 4, 4, 5] => ['0.77', '3.48', '5.03', '6.97', '8.52', '10.06', '12.00']
4: [3, 5, 3, 5, 5, 3, 7] => ['1.16', '3.10', '4.26', '6.19', '8.13', '9.29', '12.00']
4: [3, 5, 3, 5, 5, 5, 5] => ['1.16', '3.10', '4.26', '6.19', '8.13', '10.06', '12.00']
4: [3, 5, 3, 7, 3, 5, 5] => ['1.16', '3.10', '4.26', '6.97', '8.13', '10.06', '12.00']
4: [3, 5, 4, 4, 5, 5, 5] => ['1.16', '3.10', '4.65', '6.19', '8.13', '10.06', '12.00']
4: [3, 5, 5, 5, 4, 4, 5] => ['1.16', '3.10', '5.03', '6.97', '8.52', '10.06', '12.00']
3: [2, 5, 3, 5, 5, 4, 7] => ['0.77', '2.7

In [29]:
for i in [5, 6, 7, 8, 9]:
    diatonic = [s for s in scales if len(s) == i]
    print(i, len(diatonic))

5 0
6 0
7 51
8 0
9 0


# Notes

subminor and supermajor 3rds come from the 7th harmonic
neutral 3rd comes from the 11th harmonic

31-EDO captures neutral 3rds better than sub/super minor/major 3rds

# standard triadic modes of new ottoman
```
major = [3, 5, 5, 3, 5, 5, 5]
harmonic_minor = [3, 5, 5, 3, 7, 3, 5]
melodic_mior = [3, 5, 5, 5, 5, 3, 5]
harmonic_major = [5, 3, 7, 3, 5, 5, 3]
```

In the above scales 1-step interval is forbidden and only major and minor 3rds are allowed.

In general we can also allow neutral, septimal minor, and septimal major thirds. We can also allow a 1-step interval but that sounds awful.

In [12]:
def ji_to_edo(h, edo=31):
    return (edo * np.log2(h)) % edo

In [38]:
ji_to_edo(7/3, 31)

6.894165061429888

In [43]:
ji_to_edo(9/7, 31)

11.23967246092596

In [44]:
ji_to_edo(11/9, 31)

8.974705133044536

In [45]:
ji_to_edo(27/11, 31)

9.15913238931131

# TODO

- [X] A single function to generate all scales given allowed 3rds and allowed 2nds
- [ ] Play all modes of all scales using librosa or something.
- [X] Sort by number of perfect 5ths
- [ ] Find all scales where pitch classes = `[0, 5, *, 13, 18, *, *]` (why though?)

In [11]:
def delta_to_pulse(deltas):
    pulses = sum([[1] + (k-1) * [0] for k in deltas], start=[])
    return pulses

def pulse_to_delta(pulses):
    deltas = []
    delta = 1
    
    for idx, val in enumerate(pulses[1:]):
        if val == 0:
            delta += 1
            continue
        deltas.append(delta)
        delta = 1
    deltas.append(delta)
    
    return deltas